https://doc.sagemath.org/html/en/reference/rings_numerical/sage/rings/real_mpfi.html

https://doc.sagemath.org/html/en/reference/spkg/mpfi.html#spkg-mpfi

https://doc.sagemath.org/html/en/reference/spkg/mpfr.html#spkg-mpfr

In [41]:
## Adding the .simplify_full() command may give cleaner functions to look at, but drastically increases the runtime (from 0.5 sec to more than 4 min), and, most importantly, causes completely unstable computations that ruin the precision (for unknown reasons).
import time
import numpy as np
prec = 300
RBF = RealBallField(prec)
RIF = RealIntervalField(prec)
var('x')
assume(x, 'real')
var('y')
assume(y, 'real')

n_derivs=10  # Number of derivatives to compute


W = (-x/2 + sqrt(1/27 + x^2/4))^(1/3) - (x/2 + sqrt(1/27 + x^2/4))^(1/3)
W_prime = diff(W, x)
a = 1 + W/x + W_prime
b = (3*x)/2 + W
a_derivs = [a]
a_derivs_0=[a_derivs[0].taylor(x,0)]
b_derivs = [b]
b_derivs_0=[b_derivs[0].taylor(x,0)]
for k in range(1, n_derivs):
    a_derivs.append(diff(a, x, k))
    a_derivs_0.append(a_derivs[k].taylor(x,0))
    b_derivs.append(diff(b, x, k))
    b_derivs_0.append(b_derivs[k].taylor(x,0))


phi_0 = -a_derivs[0] + b_derivs[1]/2 + a_derivs[2]/2 - a_derivs[4]/2 + a_derivs[6]/2
phi_1 = -a_derivs[0] - b_derivs[1]/2 + 2*a_derivs[2] + b_derivs[3]/2 - 3*a_derivs[4] - b_derivs[5]/2
phi_2 = -a_derivs[0] - (3/2)*b_derivs[1] + (9/2)*a_derivs[2] + (5/2)*b_derivs[3] 
phi_3 = -a_derivs[0] - (5/2)*b_derivs[1] # corresponds to the sign in the inequality
phi_list = [phi_0, phi_1, phi_2, phi_3]

phi_0_0 = -a_derivs_0[0] + b_derivs_0[1]/2 + a_derivs_0[2]/2 - a_derivs_0[4]/2 + a_derivs_0[6]/2
phi_1_0 = -a_derivs_0[0] - b_derivs_0[1]/2 + 2*a_derivs_0[2] + b_derivs_0[3]/2 - 3*a_derivs_0[4] - b_derivs_0[5]/2
phi_2_0 = -a_derivs_0[0] - (3/2)*b_derivs_0[1] + (9/2)*a_derivs_0[2] + (5/2)*b_derivs_0[3]
phi_3_0 = -a_derivs_0[0] - (5/2)*b_derivs_0[1]  
phi_list_0 = [phi_0_0, phi_1_0, phi_2_0, phi_3_0]


# Base Fourier function
f = (pi/4) * exp(-2*pi*abs(x)) * (2*(pi**2)*(x**2) + 3*pi*abs(x) + 3/2)
lambda_0 = f
lambda_1 = -diff(f, x)
lambda_2 = -diff(f, x, 2)
lambda_3 = 2*diff(f, x, 3)
lambda_4 = diff(f, x, 4)
lambda_list = [lambda_0,lambda_1,lambda_2,lambda_3,lambda_4]
lambda_list_0 = [limit(lambda_0,x=0),limit(lambda_1,x=0),limit(lambda_2,x=0),limit(lambda_3,x=0),limit(lambda_4,x=0)]


M_0 = (lambda_list[0](x=x - y) * phi_list[0](x=y))
M_1 = (lambda_list[1](x=x - y) * diff(phi_list[1], x)(x=y) + lambda_list[2](x=x - y) * phi_list[1](x=y))
M_2 = (lambda_list[3](x=x - y) * diff(diff(phi_list[2], x),x)(x=y) +lambda_list[3](x=x - y) * diff(phi_list[2], x)(x=y)+ lambda_list[4](x=x - y) * phi_list[2](x=y))
M_list=[M_0,M_1,M_2]

def smooth_fast_callable(f,x,zero_value=RBF(0)):
    f_fast = fast_callable(f, vars=[x], domain=RBF)
    def f_modified(ball):
        return RBF(zero_value) if abs(RBF(ball).center()) <= 2**(-prec) else f_fast(ball) # Set the sensibility to be equal to the field precision
    return f_modified

# Convert symbolic functions to numerical ones evaluable on RealBallField elements
a_funcs = [smooth_fast_callable(a_derivs[k],x,a_derivs_0[k]) for k in range(n_derivs)]
b_funcs = [smooth_fast_callable(b_derivs[k],x,b_derivs_0[k]) for k in range(n_derivs)]
phi_funcs = [smooth_fast_callable(phi_list[k],x,phi_list_0[k]) for k in range(4)]
lambda_funcs = [smooth_fast_callable(lambda_list[k],x,lambda_list_0[k]) for k in range(5)]

phi_1_d1_0=diff(phi_list[1],x).taylor(x,0)
phi_1_d1_func=smooth_fast_callable(diff(phi_list[1],x),x,phi_1_d1_0)

phi_2_d1_0=diff(phi_list[2],x).taylor(x,0)
phi_2_d1_func=smooth_fast_callable(diff(phi_list[2],x),x,phi_2_d1_0)
phi_2_d2_0=diff(diff(phi_list[2],x),x).taylor(x,0)
phi_2_d2_func=smooth_fast_callable(diff(diff(phi_list[2],x),x),x,phi_2_d2_0)


def M_0_func(ball1,ball2):
   return lambda_funcs[0](ball1-ball2)*phi_funcs[0](ball2)
    
def M_1_func(ball1, ball2):
    return (lambda_funcs[1](ball1-ball2) * phi_1_d1_func(ball2) + lambda_funcs[2](ball1-ball2) * phi_funcs[1](ball2))

def M_2_func(ball1, ball2):
    return (lambda_funcs[3](ball1-ball2) * phi_2_d2_func(ball2) + lambda_funcs[3](ball1-ball2) * phi_2_d1_func(ball2) + lambda_funcs[4](ball1-ball2) * phi_funcs[2](ball2))


M_funcs=[M_0_func,M_1_func,M_2_func]


In [2]:
def smooth_fast_callable_rif2(f, x, zero_value=RIF(0)):
    f_fast = fast_callable(f, vars=[x], domain=RIF)
    def f_modified2(interval):
        interval = RIF(interval)
        if interval.contains_zero():
            if interval.absolute_diameter() <= 2**(-prec+1):
                return RIF(zero_value)
            else :
                a, b = interval.endpoints()
                mid = interval.center()
                left = f_modified2(RIF(a, mid))
                right = f_modified2(RIF(mid, b))
                return left.union(right)
        r = f_fast(interval)
        if r.is_NaN():
            print(interval)
        return r

    return f_modified2


In [30]:
prec = 200
RBF = RealBallField(prec)
RIF = RealIntervalField(prec)

W = (-x/2 + sqrt(1/27 + x^2/4))^(1/3) - (x/2 + sqrt(1/27 + x^2/4))^(1/3)

I = RIF(3,4)
print(abs(W(x=I)))
print(RIF(W(x=I)).upper())
print(RIF(W(x=I)).lower())

print(smooth_fast_callable_rif2(exp(x),x,RIF(1))(RIF(-1,1)))

abs(-(([2.6193722742502851633697311648546712249903633703824707853466096 .. 3.4801021696368500599192513408493492260966166125732665843351423])*sqrt(1/3) + [1.5000000000000000000000000000000000000000000000000000000000000 .. 2.0000000000000000000000000000000000000000000000000000000000000])^(1/3) + (([2.6193722742502851633697311648546712249903633703824707853466096 .. 3.4801021696368500599192513408493492260966166125732665843351423])*sqrt(1/3) + [-2.0000000000000000000000000000000000000000000000000000000000000 .. -1.5000000000000000000000000000000000000000000000000000000000000])^(1/3))
NaN
NaN
[0.36787944117144232159552377016146086744581113103176783450783671 .. 2.7182818284590452353602874713526624977572470936999595749669680]


In [36]:
prec = 200
RIF = RealIntervalField(prec)
x = var('x')

# Définir l'expression symbolique
W_expr = (-x/2 + sqrt(1/27 + x^2/4))^(1/3) - (x/2 + sqrt(1/27 + x^2/4))^(1/3)

# Transformer l'expression en fonction numérique sur le champ d'intervalles
W_interval_func = fast_callable(W_expr, vars=[x], domain=RIF)

# Créer un intervalle d'entrée
I = RIF(-1, 1)

# Évaluer et afficher la valeur absolue sur l'intervalle
print(abs(W_interval_func(I)))


[.. NaN ..]


In [19]:
prec = 200
CIF = ComplexIntervalField(prec)
x = var('x')

# même expression, mais convertie dans le domaine complexe
W = (-x/2 + sqrt(1/27 + x^2/4))^(1/3) - (x/2 + sqrt(1/27 + x^2/4))^(1/3)

I = CIF(3, 4)  # intervalle complexe (imaginaire nul)

print(W(x=I))


-(([2.6057920014084605492574776345257664159533764367361792905833627 .. 2.6057920014084605492574776345257664159533764367361792905833852] + [3.4538443571610460388485010851238979833563685012966796757249421 .. 3.4538443571610460388485010851238979833563685012966796757249745]*I)*sqrt(1/3) + [1.5000000000000000000000000000000000000000000000000000000000000 .. 1.5000000000000000000000000000000000000000000000000000000000000] + [2.0000000000000000000000000000000000000000000000000000000000000 .. 2.0000000000000000000000000000000000000000000000000000000000000]*I)^(1/3) + (([2.6057920014084605492574776345257664159533764367361792905833627 .. 2.6057920014084605492574776345257664159533764367361792905833852] + [3.4538443571610460388485010851238979833563685012966796757249421 .. 3.4538443571610460388485010851238979833563685012966796757249745]*I)*sqrt(1/3) + [-1.5000000000000000000000000000000000000000000000000000000000000 .. -1.5000000000000000000000000000000000000000000000000000000000000] - [2.0000000000

In [21]:
prec = 200
CIF = ComplexIntervalField(prec)
x = var('x')

# Définition de l'expression dans le domaine symbolique
W = (-x/2 + sqrt(1/27 + x^2/4))^(1/3) - (x/2 + sqrt(1/27 + x^2/4))^(1/3)

# Intervalle complexe [3, 4]
I = CIF(3, 4)

# Substitution manuelle de x dans l'expression
W_val = W.subs(x=I)

# Conversion de l'expression symbolique en valeur numérique (CIF)
W_num = CIF(W_val)

# Extraction de la partie réelle
re_part = W_num.real()

# Affichage des bornes
print("Inf:", re_part.lower())
print("Sup:", re_part.upper())


Inf: -1.4432102683163446529288744884710217078765672764826473363123
Sup: -1.4432102683163446529288744884710217078765672764826473363119


In [4]:
from sage.rings import real_mpfi
RIF = RealIntervalField(prec)
real_mpfi.printing_style = 'brackets'
x = var('x')
f = fast_callable(x+1,vars=[x],domain=RIF)
I = RIF(-10, 2)
# print(f(I))
# print(f(f(I)))

#print(smooth_fast_callable_rif(phi_0,x,phi_0_0)(RIF(0,0)))


I1=RIF(-2**(-prec-999999),2**(-prec-999999))
I2=RIF(-2**(-prec-9999999),2**(-prec-9999999))
print(I1)
print(I2)
print(I2 in I1)
print(I1 in I2)

[-1.2570914763098893821294359732208888884060382342032850944501169e-301090 .. 1.2570914763098893821294359732208888884060382342032850944501169e-301090]
[-1.3752797580758236428281328214757553809823827575592297065276754e-3010360 .. 1.3752797580758236428281328214757553809823827575592297065276754e-3010360]
True
False


In [5]:
# Tester avec une fonction continue tout ok, mais qui atteint son max pas à entre 0 et 2**(-prec)

f= M_2_func
Ix=RIF([-2,2])
Iy=RIF([-2,2])
print(f(Ix,Iy).upper())
print(f(Ix,Iy).lower())
print(RIF(f(Ix,Iy)).upper()-RIF(f(Ix,Iy)).lower())

94101.053071474046878292952838356448992692467288133059595200
94101.053071474046878292952838356448992692467288133059595198
9.7879567019977867900654411638848213335517463871034498216380e-55
